In [21]:
import tensorflow as tf
from tensorflow import keras
from Maze import Maze
from time import time
import numpy as np

In [22]:
optimizer = keras.optimizers.RMSprop()
loss_fn = keras.losses.binary_crossentropy

In [55]:
def train_reinforcement_network(n_layers, n_neurons, thresh=0.01, maze_height=5, maze_width=10):
    if maze_height == 'random' and maze_width == 'random':
        import random
        maze_height = random.randint(5,30)
        maze_width = random.randint(5,30)

    network_layers = [keras.layers.Dense(Maze.N_INPUTS, activation="elu", input_shape=[Maze.N_INPUTS])]
    for i in range(n_layers):
        network_layers.append(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
        network_layers.append(keras.layers.BatchNormalization())
    network_layers.append(keras.layers.Dense(1, activation="sigmoid"))
    model = keras.models.Sequential(network_layers)


    loss = 1

    step_counter = 1000
    while step_counter > 20:
        my_maze = Maze(maze_height, maze_width)
        my_maze.create_maze()

        step_counter = 0

        start_time = time()
        while not my_maze.is_maze_complete():
            with tf.GradientTape() as tape:
                options = tf.Variable([[int(x) for x in my_maze.get_maze_options()]])
                probas = model(options)
                logits = tf.math.log(probas + tf.keras.backend.epsilon())
                action = tf.random.categorical(logits, num_samples=1)
                loss = tf.reduce_mean(loss_fn(action, probas))

            step_counter += 1

            my_maze.take_action(action[0][0])
        print(f'Training time: {time()-start_time}')

In [ ]:
train_reinforcement_network(5, 100, thresh=0.4, maze_height=5, maze_width=10)